In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

# Data Cleaning

In [ ]:
df = pd.read_csv("/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df1 = df.drop(['area_type','society','balcony','availability'],axis = 1)

In [ ]:
df1.isna().sum()

In [ ]:
df1.info()

In [ ]:
df2 = df1.dropna()

In [ ]:
df2.isna().sum()

In [ ]:
df2['size'].unique()

In [ ]:
df2['size'].value_counts()

In [ ]:
df2['bhk'] = df2['size'].apply(lambda x : int(x.split(' ')[0]))

In [ ]:
df2['total_sqft'].unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df2[~df2['total_sqft'].apply(is_float)].head()

In [ ]:
def convert(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return(float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
convert('333 - 333')

In [ ]:
df3 = df2.copy()
df3['total_sqft']  =  df2['total_sqft'].apply(convert)

In [ ]:
df3.head(10)

In [ ]:
df3.isna().sum()

In [ ]:
df3 = df3.dropna()

# Feature Engineering

In [ ]:
df4 = df3.copy()

In [ ]:
df4['price_per_sqrt'] = df4['price']*100000/df4['total_sqft']

In [ ]:
df4.location.unique()

In [ ]:
len(df4.location.unique())

In [ ]:
df4.location = df4.location.apply(lambda x : x.strip())

In [ ]:
df4.location.value_counts()

In [ ]:
loc_state = df4.groupby('location')['location'].agg('count').sort_values(ascending=False)

In [ ]:
len(loc_state[loc_state<=10])

In [ ]:
loc_state_less_10 = loc_state[loc_state<=10]

In [ ]:
loc_state_less_10

In [ ]:
df4.location = df4.location.apply(lambda x: 'other' if x in loc_state_less_10 else x)

In [ ]:
df4.location.unique()

In [ ]:
df4.head()

In [ ]:
# remove outliers
df5  = df4[~(df4.total_sqft/df4.bhk < 300)]

In [ ]:
df5.price_per_sqrt.describe()

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqrt)
        st = np.std(subdf.price_per_sqrt)
        reduced_df = subdf[(subdf.price_per_sqrt>(m-st)) & (subdf.price_per_sqrt<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df6 = remove_pps_outliers(df5)
df6.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+', color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df6,"Rajaji Nagar")

In [ ]:
plot_scatter_chart(df6,"Hebbal")

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqrt),
                'std': np.std(bhk_df.price_per_sqrt),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqrt<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df7 = remove_bhk_outliers(df6)
df7.shape

In [ ]:
df8 = df7.copy()
plot_scatter_chart(df8,"Rajaji Nagar")

In [ ]:
plot_scatter_chart(df8,"Hebbal")

In [ ]:


import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df8.price_per_sqrt,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")



In [ ]:
df8.bath.unique()

In [ ]:


plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")



In [ ]:
df8[df8.bath>10]

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:


df9 = df8[df8.bath<df8.bhk+2]
df9.shape



In [ ]:


df10 = df9.drop(['size','price_per_sqrt'],axis='columns')
df10.head(3)



In [ ]:


dummies = pd.get_dummies(df10.location)
dummies.head(3)



In [ ]:


df11 = pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df11.head()



In [ ]:


df12 = df11.drop('location',axis='columns')
df12.head(2)



# Build a Model Now...


In [ ]:
x = df12.drop(['price'],axis='columns')
y = df12.price

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=10)

In [ ]:


from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)




# Use K Fold cross validation to measure accuracy of our LinearRegression model


In [ ]:


from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), x, y, cv=cv)

